In [124]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import string 
import time 
import nfl_data_py as nfl
import re
import lxml

## Recommended data fields (per player)
- Identity: `player_id`, `name`, `position`, `draft_year`
- Draft capital: `draft_round`, `draft_pick_overall`, `team`
- College production (final season + career): rushing/receiving/passing as appropriate
- Context: `conference`, `age_on_draft_day`, `height_in`, `weight_lb`, `forty_yd`, etc.
- NFL rookie outcomes (for training): rushing/receiving/passing/fumbles/INT, games played

## Get Draft Data 
Download Excel files for the drafts of the last 5 years from https://www.pro-football-reference.com/

These are the fields of the above datasets:
- Rnd -- Round selected in draft
- Supplemental draft round indicated with 'S'
- Pick -- Overall Selection in Draft
- Pos -- Position
- Age -- Age as of September 1 of the draft year
- To -- Last Year
- Misc
- AP1 -- First-team all-pro selections
- PB -- Pro Bowl Selections
- St -- Number of years as primary starter for his team at his position
- Approx Val
- wAV -- Weighted Career Approximate Value.
- See the glossary for details on how we compute Approximate Value.
- The career AV is computed by summing
- 100 percent of the AV of his best season,
- 95 percent of the AV of his next-best season,
- 90 percent of the AV of his third-best season,
- and so on
- DrAV -- Weighted AV accumulated for team that drafted this player
- Games
- G -- Games played
#### Passing
- Cmp -- Passes completed
- Att -- Passes attempted
- Yds -- Yards Gained by Passing (For teams, sack yardage is deducted from this total)
- TD -- Passing Touchdowns
- Int -- Interceptions thrown
#### Rushing
- Att -- Rushing Attempts (sacks not included in NFL)
- Yds -- Rushing Yards Gained (sack yardage is not included by NFL)
- TD -- Rushing Touchdowns
#### Receiving
- Rec -- Receptions
- Yds -- Receiving Yards
- TD -- Receiving Touchdowns
#### Tackles
- Solo -- Tackles
- Before 1994: unofficial and inconsistently recorded from team to team. For amusement only.
- 1994-now: unofficial but consistently recorded.
#### Def Interceptions
- Int -- Passes intercepted on defense
- Sk -- Sacks (official since 1982, based on play-by-play, game film, and other research since 1960)

In [208]:
# Get Drafted College Football Players from the last 5 years 

df_2024 = pd.read_csv("2024_draft.csv")
df_2024["draft_year"] = 2024
df_2023 = pd.read_csv("2023_draft.csv")
df_2023["draft_year"] = 2023
df_2022 = pd.read_csv("2022_draft.csv")
df_2022["draft_year"] = 2022
df_2021 = pd.read_csv("2021.csv")
df_2021["draft_year"] = 2021
df_2020 = pd.read_csv("2020_draft.csv")
df_2020["draft_year"] = 2020
df_2019 = pd.read_csv("2019_draft.csv")
df_2019["draft_year"] = 2019


drafts_df = pd.concat([df_2020, df_2021, df_2022, df_2023, df_2024], axis=0)

for name in drafts_df["Player"]:
    if isinstance(name, str):  # Only process if it's a string
        clean_name = re.sub(r'[^\w\s-]', '', name.lower())  # remove punctuation 
        drafts_df.loc[drafts_df["Player"] == name, "url_name"] = clean_name.replace(" ", "-")

print(len(drafts_df))
print(drafts_df[:5])



1297
   Rnd  Pick   Tm          Player Pos   Age      To  AP1   PB   St  ...  \
0  1.0   1.0  CIN      Joe Burrow  QB  23.0  2024.0  0.0  2.0  5.0  ...   
1  1.0   2.0  WAS     Chase Young  DE  21.0  2024.0  0.0  1.0  1.0  ...   
2  1.0   3.0  DET     Jeff Okudah  CB  21.0  2024.0  0.0  0.0  2.0  ...   
3  1.0   4.0  NYG   Andrew Thomas   T  21.0  2024.0  0.0  0.0  5.0  ...   
4  1.0   5.0  MIA  Tua Tagovailoa  QB  22.0  2024.0  0.0  1.0  5.0  ...   

   Unnamed: 20  Receiving  Unnamed: 22  Unnamed: 23  Unnamed: 24  Unnamed: 25  \
0          NaN        NaN          NaN          NaN          NaN          NaN   
1          NaN        NaN          NaN          NaN          NaN          NaN   
2          NaN        NaN          NaN          NaN          NaN          NaN   
3          NaN        NaN          NaN          NaN          NaN          NaN   
4          NaN        NaN          NaN          NaN          NaN          NaN   

   Unnamed: 26  Unnamed: 27  Unnamed: 28        url_name 

## Or use NFL_data_py to import draft info

In [218]:
years = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

drafts_df = nfl.import_draft_picks(years)

In [219]:
# Filter for relevant positions
positions = ['QB', 'TE', 'WR', 'RB']

drafts_df = drafts_df[drafts_df["position"].isin(positions)]
print(len(drafts_df))

868


In [ ]:
# Add draft info to master df 
df = pd.DataFrame()
df["name"], df["position"], df["team"], df["draft_year"], df["draft_round"], df["draft_pick_overall"], df["age_on_draft_day"] = drafts_df["pfr_player_name"], drafts_df["position"], drafts_df["team"], drafts_df["season"], drafts_df["round"], drafts_df["pick"], drafts_df["age"]
print(df[:2])


               name position team  draft_year  draft_round  \
9595  Blake Bortles       QB  JAX        2014            1   
9596  Sammy Watkins       WR  BUF        2014            1   

      draft_pick_overall  age_on_draft_day  
9595                   3              22.0  
9596                   4              21.0  


## College Football Stats: Web Scraping

In [ ]:
# If Webscraping: 
df["url_name"] = drafts_df["url_name"] # MAKE SURE ORDER and length OF NAMES IS THE SAME!

In [132]:
def get_player_stats(player_names):
    all_players_data = {}
    for name in player_names:
        found = False 
        for page in range(1,10): 
            url = f"https://www.sports-reference.com/cfb/players/{name}-{page}.html"
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    print(f"Found data for {name} at {url}")
                    soup = BeautifulSoup(response.text, "html.parser")

                    # Get tables 
                    tables = pd.read_html(response.text)

                    # There are mulitple tables per page, map them to their html ids
                    table_map = {}
                    for i, table in enumerate(soup.find_all("table")):
                        table_id = table.get("id", f"table_{i}")
                        table_map[table_id] = tables[i]

                    # Filter only the tables we care about
                    relevant_ids = ["passing_standard", "receiving_standard", "punt_return_standard"]
                    player_tables = {tid: df for tid, df in table_map.items() if tid in relevant_ids}

                    all_players_data[name] = player_tables

                    found = True
                    break
            except Exception as e:
                print(f"Error fetching {url}: {e}")
        if not found:
            print(f"No page found for {name}")

    return all_players_data 

In [133]:
player_names = df["url_name"]
college_stats = get_player_stats(["joe-burrow"])
# print(college_stats["bryce-young"]["tables"]["passing_standard"].head())
# print(college_stats["bryce-young"]["height_in"])

Found data for joe-burrow at https://www.sports-reference.com/cfb/players/joe-burrow-1.html
Error fetching https://www.sports-reference.com/cfb/players/joe-burrow-1.html: lxml not found, please install it


KeyboardInterrupt: 

In [ ]:
def get_nfl_pages(years):
    nfl_player_pages = []

    # Add headers to mimic a real browser -- site was blocking scraping 
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }

    for year in years:
        url = f"https://www.pro-football-reference.com/years/{year}/draft.htm"
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                print(f"Found data for {year} draft at {url}")
                soup = BeautifulSoup(response.text, "html.parser")

                # Get tables 
                tables = pd.read_html(response.text)

                # Get player nfl stats pages 
                base_url = "https://www.pro-football-reference.com/"

                container = soup.find("div", {"id": "div_drafts"})
                if container:
                    table = container.find("table")
                    if table:
                        tbody = table.find("tbody")
                        if tbody:
                            rows = tbody.find_all("tr")
                            for row in rows:
                                player_cell = row.find("td", {"data-stat": "player"})
                                if player_cell:
                                    link_tag = player_cell.find("a")
                                    if link_tag and "href" in link_tag.attrs:
                                        relative_url = link_tag["href"]
                                        full_url = base_url + relative_url
                                        player_name = link_tag.get_text(strip=True)
                                        nfl_player_pages.append({"name": player_name, "url": full_url})
                        else:
                            print(f"No tbody found for {year}")
                    else:
                        print(f"No table found in div_drafts for {year}")
                else:
                    print(f"No div_drafts container found for {year}")
            else:
                print(f"HTTP {response.status_code} for {url}")
                
        except Exception as e:
            print(f"Error fetching {url}: {e}")
    return nfl_player_pages

In [78]:
years = ["2023", "2024"]
nfl_pages = get_nfl_pages(["2023"])
print(nfl_pages)

HTTP 403 for https://www.pro-football-reference.com/years/2023/draft.htm
[]


In [ ]:
def get_nfl_stats(nfl_player_pages):
    all_nfl_data = {}

    # Add headers to help with blocking
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    for entry in nfl_player_pages:
        name = entry["name"]
        url = entry["url"]
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                print(f"Found data for {name} at {url}")
                soup = BeautifulSoup(response.text, "html.parser")

                # Get tables 
                tables = pd.read_html(response.text)
                # There are mulitple tables per page, map them to their html ids
                table_map = {}
                for i, table in enumerate(soup.find_all("table")):
                    table_id = table.get("id", f"table_{i}")
                    table_map[table_id] = tables[i]

                # Filter only the tables we care about
                relevant_ids = ["passing", "rushing_and_receiving"]
                player_tables = {tid: df for tid, df in table_map.items() if tid in relevant_ids}

                all_nfl_data[name] = player_tables

                # Add delay to be respectful
                time.sleep(2)
                
            else:
                print(f"HTTP {response.status_code} for {name} at {url}")
        except Exception as e:
            print(f"Error fetching {url}: {e}")
    # Check for any data at all
    if not all_nfl_data:
        print("No data found for any players - likely blocked by site")
    else:
        print(f"Successfully retrieved data for {len(all_nfl_data)} players")

    return all_nfl_data 
        

In [80]:
urls = [{"name": "Bryce Young", "url": "https://www.pro-football-reference.com/players/Y/YounBr01.htm"}]
rookie_data = get_nfl_stats(urls)

No page found for Bryce Young


## Get Rookie Season Data: Using NFL_data_py package
https://pypi.org/project/nfl-data-py/

option: 
nfl.import_seasonal_data(years, s_type)
Returns seasonal data, including various calculated market share stats specific to receivers

years (List[int]) : required, list of years to pull data for (earliest available is 1999)

s_type (str) : optional (default 'REG') season type to include in average ('ALL','REG','POST')


nfl.import_seasonal_pfr(s_type, years)
Returns a dataframe of season-aggregated data sourced from players' pages on pro-football-reference.com. E.g. Patrick Mahomes

s_type (str) : required, the type of stat data to request. Must be one of pass, rec, or rush.

years (List[int]) : optional, years to return data for

In [197]:
def get_player_id_map(names: list[str], years: list[int]) -> dict:
    """
    Build a mapping of player_id -> 'First Last' given player names and corresponding years.
    
    """
    if len(names) != len(years):
        raise ValueError("names and years must have the same length")
    
    player_map = {}
    
    # Loop through each (name, year) pair
    for name, year in zip(names, years):
        roster = nfl.import_seasonal_rosters([year])
        #roster["player_name"] = roster["first_name"] + " " + roster["last_name"]
        print(roster["height"])
        match = roster[roster["player_name"] == name]
        
        if not match.empty:
            player_id = match.iloc[0]["player_id"]
            player_map[player_id] = {
                "name": name,
                "rookie_year": year,
                "position": match.iloc[0]["position"],
                "team": match.iloc[0]["team"],
                "height_in": match.iloc[0].get("height"),
                "weight_lb": match.iloc[0].get("weight"),
                "age_on_draft_day": match.iloc[0].get("age_on_draft_day")
            }
        else:
            print(f"No match found for {name} in {year}")
    
    return player_map


In [221]:
player_map = get_player_id_map(df["name"], df["draft_year"])

0       71.0
1       76.0
2       77.0
3       72.0
4       73.0
        ... 
2147    71.0
2148    74.0
2149    76.0
2150    74.0
2151    71.0
Name: height, Length: 2152, dtype: float64
0       71.0
1       76.0
2       77.0
3       72.0
4       73.0
        ... 
2147    71.0
2148    74.0
2149    76.0
2150    74.0
2151    71.0
Name: height, Length: 2152, dtype: float64
0       71.0
1       76.0
2       77.0
3       72.0
4       73.0
        ... 
2147    71.0
2148    74.0
2149    76.0
2150    74.0
2151    71.0
Name: height, Length: 2152, dtype: float64
0       71.0
1       76.0
2       77.0
3       72.0
4       73.0
        ... 
2147    71.0
2148    74.0
2149    76.0
2150    74.0
2151    71.0
Name: height, Length: 2152, dtype: float64
0       71.0
1       76.0
2       77.0
3       72.0
4       73.0
        ... 
2147    71.0
2148    74.0
2149    76.0
2150    74.0
2151    71.0
Name: height, Length: 2152, dtype: float64
No match found for Odell Beckham Jr. in 2014
0       71.0
1       76.0

In [70]:
# Get player stats for specific season
def get_rookie_stats(player_info_map: dict):
    all_stats = []
    
    for player_id, info in player_info_map.items():
        rookie_year = info["rookie_year"]
        seasonal_stats = nfl.import_seasonal_data([rookie_year])
        
        player_stats = seasonal_stats[seasonal_stats["player_id"] == player_id].copy()
        player_stats["player_name"] = info["name"]
        player_stats["position"] = info["position"]
        player_stats["team"] = info["team"]
        all_stats.append(player_stats)
    
    # Combine all players into a single DataFrame
    return pd.concat(all_stats, ignore_index=True)

In [224]:
rookie_stats = get_rookie_stats(player_map)

print(rookie_stats[:5])

    player_id  season season_type  completions  attempts  passing_yards  \
0  00-0031407    2014         REG          280       475         2908.0   
1  00-0031325    2014         REG            0         0            0.0   
2  00-0031408    2014         REG            0         0            0.0   
3  00-0031387    2014         REG            0         0            0.0   
4  00-0031236    2014         REG            0         0            0.0   

   passing_tds  interceptions  sacks  sack_yards  ...    rtd_sh    rfd_sh  \
0           11           17.0   55.0       345.0  ...  0.000000  0.000000   
1            0            0.0    0.0         0.0  ...  0.260870  0.252688   
2            0            0.0    0.0         0.0  ...  0.631579  0.308725   
3            0            0.0    0.0         0.0  ...  0.062500  0.092715   
4            0            0.0    0.0         0.0  ...  0.157895  0.155280   

   rtdfd_sh       dom     w8dom    yptmpa    ppr_sh    player_name  position  \
0  0.0

In [28]:
# see what columns are available
seasonal_stats = nfl.import_seasonal_data([2023])
print("Available columns:")
print(seasonal_stats.columns.tolist())
roster_data = nfl.import_seasonal_rosters([2023])
print(roster_data.columns.tolist())

Available columns:
['player_id', 'season', 'season_type', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share', 'wopr_x', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr', 'games', 'tgt_sh', 'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh', 'rfd_sh', 'rtdfd_sh', 'dom', 'w8dom', 'yptmpa', 'ppr_sh']
['season', 'team', 'position', 'depth_chart_positi

## Fantasy Scoring 
Assumptions for Half-PPR + TE Premium: 
- Passing TD: 4 points
- Passing Yards: 1 point per 25 yards
- Interceptions: -2 points
- Rushing TD: 6 points
- Rushing Yards: 1 point per 10 yards
- Receptions: 0.5 points (Half PPR)
- Receiving TD: 6 points
- Receiving Yards: 1 point per 10 yards
- Fumbles Lost: -2 points
- Tight End Premium: +1 point per reception if the player is a TE

In [26]:
def calculate_halfppr_tep(stats: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate Half-PPR fantasy points with Tight End Premium.

    Args:
        stats: Seasonal stats including player_id, position, receptions, etc.

    Returns:
        pd.DataFrame: Original DataFrame with new 'fantasy_points_halfppr_tep' column
    """
    df = stats.copy()
    
    # Base fantasy points
    df['fantasy_points_halfppr_tep'] = (
        df.get('passing_yards', 0) / 25 +
        df.get('passing_tds', 0) * 4 +
        df.get('interceptions', 0) * -2 +
        df.get('rushing_yards', 0) / 10 +
        df.get('rushing_tds', 0) * 6 +
        df.get('receiving_yards', 0) / 10 +
        df.get('receiving_tds', 0) * 6 +
        df.get('receptions', 0) * 0.5 +
        df.get('rushing_fumbles_lost', 0) * -2 +
        df.get('receiving_fumbles_lost', 0) * -2
    )
    
    # Tight End Premium: +1 point per reception for TEs
    if 'position' in df.columns:
        df.loc[df['position'] == 'TE', 'fantasy_points_halfppr_tep'] += df.loc[df['position'] == 'TE', 'receptions']
    
    return df


In [225]:
rookie_fnt_pts = calculate_halfppr_tep(rookie_stats)
print(rookie_fnt_pts[['player_name', 'position', 'fantasy_points_halfppr_tep']])

            player_name position  fantasy_points_halfppr_tep
0         Blake Bortles       QB                      166.22
1         Sammy Watkins       WR                      165.50
2            Mike Evans       WR                      211.10
3            Eric Ebron       TE                       68.30
4         Brandin Cooks       WR                      112.80
..                  ...      ...                         ...
642  Jordan Whittington       WR                       41.50
643       Ryan Flournoy       WR                       13.20
644         Jaheim Bell       TE                        5.00
645       Devaughn Vele       WR                       86.00
646          Devin Culp       TE                       16.30

[647 rows x 3 columns]


## Add Rookie Data to master data frame  

In [ ]:
def add_rookie_stats_to_master(master_df: pd.DataFrame, rookie_stats: pd.DataFrame) -> pd.DataFrame:
    """
    Only adds the following stats:
    Rec, RushYds, RecYds, PassYds, RushTD, RecTD, PassTD, Fumbles, INT

    """
    # Select relevant columns from rookie stats
    rookie_columns_map = {
        'receptions': 'Rec',
        'rushing_yards': 'RushYds',
        'receiving_yards': 'RecYds',
        'passing_yards': 'PassYds',
        'rushing_tds': 'RushTD',
        'receiving_tds': 'RecTD',
        'passing_tds': 'PassTD',
        'rushing_fumbles_lost': 'Fumbles',
        'receiving_fumbles_lost': 'Fumbles',  # Combine fumbles into single column
        'interceptions': 'INT'
    }

    # Copy rookie stats to avoid modifying original
    rookie_copy = rookie_stats.copy()

    # Combine fumbles from rushing and receiving
    rookie_copy['Fumbles'] = rookie_copy.get('rushing_fumbles_lost', 0) + rookie_copy.get('receiving_fumbles_lost', 0)

    # Build new DataFrame with player_id + mapped columns
    rookie_for_merge = pd.DataFrame({'player_id': rookie_copy['player_id']})
    
    for col, new_col in rookie_columns_map.items():
        if col in rookie_copy.columns:
            rookie_for_merge[new_col] = rookie_copy[col]

    # Drop duplicates in case a stat was counted twice (like fumbles)
    rookie_for_merge = rookie_for_merge.groupby('player_id', as_index=False).sum()

    # Merge rookie stats into master df on player_id
    merged_df = master_df.merge(rookie_for_merge, on='player_id', how='left')

    return merged_df


In [239]:
# Save a copy of rookie stats 
rookie_fnt_pts_2014_2024 = rookie_fnt_pts

## College Stats: Using CFBD API
https://api.collegefootballdata.com/#/

##### Offensive stats

YDS → Yards (context matters: Rush YDS, Pass YDS, Rec YDS)

TD → Touchdowns

REC → Receptions

CAR → Carries (rushing attempts)

ATT → Attempts (passing or rushing, depending on table)

COMPLETIONS → Completed passes

YPA → Yards Per Attempt (passing or rushing)

YPC → Yards Per Carry (rushing efficiency)

YPR → Yards Per Reception

AVG → Average yards (depends on stat: could be YPC, YPR, punt avg, etc.)

LONG → Longest play (longest rush, catch, etc.)

PTS → Points scored (rarely used in player stats, more for kickers)

#### Passing / kicking

XPA → Extra Point Attempts

XPM → Extra Points Made

FGM → Field Goals Made

FGA → Field Goal Attempts

PCT → Completion % (QBs) OR FG % (kickers)

YPP → Yards Per Punt

#### Defensive stats

SOLO → Solo tackles

TOT → Total tackles (solo + assists)

TFL → Tackles For Loss

SACKS → Quarterback sacks

PD → Passes Defended (knockdowns/deflections)

INT → Interceptions

QB HUR → QB Hurries (pressures)

FUM → Fumbles forced/recovered (depends on context)

LOST → Fumbles lost (by offense)

#### Special teams

NO → Number of attempts (e.g., punt attempts, returns)

TB → Touchbacks (kickoffs)

In 20 → Kicks/punts inside the 20-yard line

In [226]:

API_KEY = "jhx47mf3+GY8ZsAD51GHBMi7THo3VtKVg6MXZ+AkXNsvS+MoM0/5qz+F3N5hjS7B"
BASE_URL = "https://api.collegefootballdata.com"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}


In [210]:
def get_player_season_stats(year: int) -> pd.DataFrame:
    """Pull season-level stats for all players in a given year."""
    url = f"{BASE_URL}/stats/player/season"
    params = {"year": year}
    try:
        # Add a small delay to avoid rate limiting
        time.sleep(0.5)
        
        r = requests.get(url, headers=HEADERS, params=params)
        r.raise_for_status()
        data = r.json()

        if not data:  # Check if data is empty
            print(f"No data returned for year {year}")
            return pd.DataFrame()
            
        df = pd.DataFrame(data)
        
    except requests.exceptions.RequestException as e:
        print(f"Request failed for year {year}: {e}")
        return pd.DataFrame()

    if df.empty:
        return pd.DataFrame()
    
    print(f"Raw columns for {year}: {df.columns.tolist()}")
    print(f"First few rows:\n{df.head()}")
    print(f"Sample statType values: {df['statType'].unique()[:10] if 'statType' in df.columns else 'No statType column'}")
    
    # Flatten schema
    df = df.rename(columns={"player": "player_name", "playerId": "player_id"})
    # Keep only columns we need
    required_cols = ["season", "player_id", "player_name", "position", "team", "conference", "category", "statType", "stat"]
    available_cols = [col for col in required_cols if col in df.columns]
    df = df[available_cols]

    # Pivot statType -> columns
    # Create a combined column for category + statType to make variables that make sense
    if "category" in df.columns and "statType" in df.columns:
        df['stat_name'] = df['category'].astype(str) + '_' + df['statType'].astype(str)
        
        print(f"Created {df['stat_name'].nunique()} unique stat combinations")
        print(f"Sample stat names: {sorted(df['stat_name'].unique())[:15]}")

        index_cols = [col for col in ["season", "player_id", "player_name", "position", "team", "conference"] if col in df.columns]
        
        print(f"Pivoting with index_cols: {index_cols}")
        print(f"Unique statTypes: {df['statType'].unique()}")
        
        df = df.pivot_table(
            index=index_cols,
            columns="stat_name",
            values="stat",
            aggfunc="sum"
        ).reset_index()

        # Convert stats to numeric
        stat_cols = df.columns.difference(index_cols)
        df[stat_cols] = df[stat_cols].apply(pd.to_numeric, errors="coerce")

        print(f"After pivot: {len(df.columns)} total columns")
        print(f"Stat columns created: {len(stat_cols)}")
        
    else:
        print("Missing statType or stat columns - cannot pivot")

    return df

In [211]:
def filter_for_fantasy_scoring(df):
    # Essential stats for fantasy point calculation
    essential_stats = [
        'season', 'player_id', 'player_name', 'position', 'team', 'conference',
        'passing_TD', 'passing_YDS', 'passing_INT',
        'rushing_TD', 'rushing_YDS',
        'receiving_REC', 'receiving_TD', 'receiving_YDS',
        'fumbles_LOST'
    ]
    
    # Optional contextual stats
    contextual_stats = [
        'passing_ATT', 'passing_COMPLETIONS', 'passing_PCT', 'passing_YPA',
        'rushing_CAR', 'rushing_YPC', 'rushing_LONG',
        'receiving_YPR', 'receiving_LONG',
        'fumbles_FUM'
    ]
    
    # Combine and filter to available columns
    all_desired_stats = essential_stats + contextual_stats
    available_cols = [col for col in all_desired_stats if col in df.columns]
    
    return df[available_cols]

In [214]:
# url = f"{BASE_URL}/stats/player/season"
# params = {"year": 2022}
# r = requests.get(url, headers=HEADERS, params=params)
# r.raise_for_status()
# results = r.json()
# print(results[:2])

In [215]:
def get_player_usage(year: int) -> pd.DataFrame:
    """Pull player usage data for a given year."""
    url = f"{BASE_URL}/player/usage"
    params = {"year": year}
    r = requests.get(url, headers=HEADERS, params=params)
    r.raise_for_status()
    df = pd.DataFrame(r.json())
    
    if df.empty:
        return pd.DataFrame()

    # Flatten usage dict
    usage_df = pd.json_normalize(df.to_dict(orient="records"), sep="_")
    usage_df = usage_df.rename(columns={"id": "player_id"})
    
    possible_usage_cols = ["player_id", "season", "usage_overall", "usage_rush", "usage_pass"]
    usage_cols = [col for col in possible_usage_cols if col in usage_df.columns]
    usage_df = usage_df[usage_cols]

    return usage_df


In [216]:
def get_college_stats_for_years(years, player_list=None):
    """
    Fetch and combine college stats for multiple years.
    """
    all_stats = []
    
    for year in years:
        print(f"Fetching college stats for year {year}...")
        
        try:
            # Get season stats
            season_stats = get_player_season_stats(year)
            if season_stats.empty:
                print(f"No season stats found for {year}")
                continue
                
            # Get usage stats
            usage_stats = get_player_usage(year)
            if usage_stats.empty:
                print(f"No usage stats found for {year}")
                # Continue with just season stats
                combined_df = season_stats.copy()
            else:
                # Merge season and usage stats
                combined_df = season_stats.merge(
                    usage_stats,
                    on=["player_id", "season"],
                    how="left"
                )
            
            combined_df["college_year"] = year
            all_stats.append(combined_df)
            
        except Exception as e:
            print(f"Error fetching data for year {year}: {e}")
            continue
    
    if not all_stats:
        print("No data retrieved for any year")
        return pd.DataFrame()
    
    # Combine all years
    final_df = pd.concat(all_stats, ignore_index=True)
    
    # Filter to specific players if provided
    if player_list is not None:
        final_df = final_df[final_df["player_name"].isin(player_list)].copy()
    
    # Fill missing values
    numeric_cols = final_df.select_dtypes(include=['float64', 'int64']).columns
    final_df[numeric_cols] = final_df[numeric_cols].fillna(0)
    
    return final_df

In [230]:
if __name__ == "__main__":
    # Example usage
    # years = [2018, 2019, 2020, 2021, 2022, 2023]
    
    # If you have a specific list of players
    # players = ["Bryce Young", "C.J. Stroud", "Anthony Richardson"]
    # college_stats = get_college_stats_for_years(years, players)
    
    # Or get all players
    # years = [2019] 
    years = [] 
    for i in range(2013, 2024):
        years.append(i) 
    print(years)
    college_stats = get_college_stats_for_years(years)
    
    print(f"Retrieved data for {len(college_stats)} player-seasons")
    print(f"Columns: {college_stats.columns.tolist()}")
    print(college_stats.head())

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
Fetching college stats for year 2013...
Raw columns for 2013: ['season', 'playerId', 'player', 'position', 'team', 'conference', 'category', 'statType', 'stat']
First few rows:
   season playerId             player position            team  \
0    2013   530417     Quinshad Davis       WR  North Carolina   
1    2013   500517  Ja'Terian Douglas       RB           Tulsa   
2    2013   530652      George Morris       RB        Syracuse   
3    2013   521344     Brandon Howard       TE   Massachusetts   
4    2013   502454       Josh Quezada       RB    Fresno State   

       conference     category statType stat  
0             ACC    receiving      YDS  730  
1  Conference USA      rushing       TD    2  
2             ACC    receiving      YDS   20  
3    Mid-American  kickReturns       TD    0  
4   Mountain West      rushing      CAR  155  
Sample statType values: ['YDS' 'TD' 'CAR' 'REC' 'In 20' 'INT' 'LONG' 'NO' 'TB

In [231]:
# Filter for relevant columns 
filtered_college_stats = filter_for_fantasy_scoring(college_stats)

In [232]:
player_names = df["name"]
drafted_players_college_stats = filtered_college_stats[filtered_college_stats["player_name"].isin(player_names)]

In [ ]:
def get_most_recent_year_per_player(df):
    """
    For each player, keep only their most recent college season in the dataset.
    """
    # Get the most recent season for each player
    most_recent = df.groupby('player_name')['season'].max().reset_index()
    most_recent.columns = ['player_name', 'most_recent_season']
    
    # Merge back to get only the most recent year for each player
    final_df = df.merge(most_recent, on='player_name')
    final_df = final_df[final_df['season'] == final_df['most_recent_season']]
    
    # Clean up
    final_df = final_df.drop('most_recent_season', axis=1)
    
    print(f"Kept most recent year for {len(final_df)} players")
    return final_df

In [233]:
latest_year_college_stats = get_most_recent_year_per_player(drafted_players_college_stats)

Kept most recent year for 826 players


## Merge College, Rookie, and Draft Stats

In [237]:
# College stats
print(latest_year_college_stats.columns)
# Draft stats
print(df.columns)
# Rookie stats and HPPR fantasy points 
print(rookie_fnt_pts.columns)



Index(['season', 'player_id', 'player_name', 'position', 'team', 'conference',
       'passing_TD', 'passing_YDS', 'passing_INT', 'rushing_TD', 'rushing_YDS',
       'receiving_REC', 'receiving_TD', 'receiving_YDS', 'fumbles_LOST',
       'passing_ATT', 'passing_COMPLETIONS', 'passing_PCT', 'passing_YPA',
       'rushing_CAR', 'rushing_YPC', 'rushing_LONG', 'receiving_YPR',
       'receiving_LONG', 'fumbles_FUM', 'conference_strength'],
      dtype='object')
Index(['name', 'position', 'team', 'draft_year', 'draft_round',
       'draft_pick_overall', 'age_on_draft_day'],
      dtype='object')
Index(['player_id', 'season', 'season_type', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_

In [238]:
# Add conference strength
conference_strength_map = {
    # Power/FBS
    "SEC": 10.0,
    "Big Ten": 9.5,
    "Big 12": 8.5,
    "ACC": 8.0,
    "Pac-12": 8.0,            # historical level (adjust per season if you want)
    "American Athletic": 6.5, # AAC
    "Mountain West": 5.5,
    "Sun Belt": 5.0,
    "Mid-American": 4.5,      # MAC
    "Conference USA": 4.0,
    "FBS Independents": 6.0,  # baseline; see note below

    # FCS (top to lower)
    "MVFC": 3.6,              # Missouri Valley
    "CAA": 3.3,               # Coastal/Colonial
    "Southern": 3.2,          # SoCon
    "OVC": 2.6,               # Ohio Valley
    "Southland": 2.5,
    "Patriot": 2.3,
    "SWAC": 2.2,
    "Ivy": 2.0,
    "Pioneer": 1.8
}

latest_year_college_stats["conference_strength"] = latest_year_college_stats["conference"].map(conference_strength_map)
 

In [240]:
# Add height and weight from player_map 
# Convert player_map (dict of dicts) → DataFrame
player_info_df = pd.DataFrame.from_dict(player_map, orient="index").reset_index()
player_info_df = player_info_df.rename(columns={"index": "player_id"})

# Merge with rookie stats df
rookie_stats = rookie_stats.merge(
    player_info_df[["player_id", "height_in", "weight_lb"]],
    on="player_id",
    how="left"
)


In [241]:
# Add columns prefixed to distinguish college from rookie stats 
rookie_fnt_pts_pref = rookie_fnt_pts.rename(
    columns={col: f"R_{col}" for col in rookie_fnt_pts.columns if col not in ["player_id", "player_name", "rookie_year", "position"]}
)

latest_year_college_stats_pref = latest_year_college_stats.rename(
    columns={col: f"C_{col}" for col in latest_year_college_stats.columns if col not in ["player_id", "player_name", "position"]}
)


In [242]:
# Merge 
df.rename(columns={"name": "player_name"}, inplace=True)
on_keys = ["player_name", "position"]

df_master = df.merge(rookie_fnt_pts_pref, on=on_keys, how="inner")
df_master = df_master.merge(latest_year_college_stats_pref, on=on_keys, how="inner")

In [243]:
# Save as .csv
df_master.to_csv('df_master_2014_2024.csv', index=False)

## Next steps: 
Include all college seasons and break-out seaon, weighted differently by position for better predictions 




In [ ]:

API_KEY = "YOUR_API_KEY"
BASE_URL = "https://api.collegefootballdata.com"

HEADERS = {"Authorization": f"Bearer {API_KEY}"}

def get_player_college_profile(player_name: str, draft_year: int):
    """
    computes final season, career totals, breakout age, and weighted career stats.
    
    Args:
        player_name (str): Full player name 
        draft_year (int): NFL draft year (college final season is draft_year - 1).
    
    """
    # Get all player season stats
    url = f"{BASE_URL}/stats/player/season"
    params = {"year": None, "team": None}  # None pulls all
    r = requests.get(url, headers=HEADERS, params=params)
    r.raise_for_status()
    df = pd.DataFrame(r.json())

    # Filter for this player
    player_df = df[df["player"].str.lower() == player_name.lower()].copy()
    if player_df.empty:
        print(f"No data found for {player_name}")
        return None

    # Convert numeric columns
    num_cols = ["stat"]
    player_df["stat"] = pd.to_numeric(player_df["stat"], errors="coerce")

    # Pivot so stats are columns
    player_df = player_df.pivot_table(
        index=["season", "player", "position", "team", "conference"],
        columns="statType",
        values="stat",
        aggfunc="sum"
    ).reset_index()

    # --- Final season stats ---
    final_season = player_df[player_df["season"] == draft_year - 1]

    # --- Career totals ---
    career_totals = player_df.drop(columns=["season", "player", "position", "team", "conference"]).sum()

    # --- Career averages (per season) ---
    career_averages = career_totals / player_df.shape[0]

    # --- Breakout age ---
    # For WR/TE: breakout if >= 20% team receiving yards OR >= 20% team receiving TDs
    breakout_age = None
    if "receivingYards" in player_df.columns and "receivingTDs" in player_df.columns:
        for _, row in player_df.iterrows():
            # Simplified dominator check (requires team stats ideally, but we use absolute threshold for now)
            if row.get("receivingYards", 0) >= 500 or row.get("receivingTDs", 0) >= 5:
                breakout_age = draft_year - row["season"]  # age could be refined w/ DOB
                break

    # --- Weighted stats (recent years weighted higher) ---
    weights = [0.6, 0.3, 0.1]  # Final, second-last, third-last
    seasons_sorted = player_df.sort_values("season", ascending=False).head(3)
    weight_sum = sum(weights[: len(seasons_sorted)])
    weighted_stats = (seasons_sorted.drop(columns=["season", "player", "position", "team", "conference"])
                      .multiply(weights[: len(seasons_sorted)][::-1], axis=0)
                      .sum()) / weight_sum

    return {
        "final_season": final_season,
        "career_totals": career_totals,
        "career_averages": career_averages,
        "breakout_age": breakout_age,
        "weighted_stats": weighted_stats
    }


In [ ]:
marvin = get_player_college_profile("Marvin Harrison Jr", 2024)

print("Final Season:")
print(marvin["final_season"])

print("\nCareer Totals:")
print(marvin["career_totals"])

print("\nWeighted Career Stats:")
print(marvin["weighted_stats"])

print("\nBreakout Age (years before draft):", marvin["breakout_age"])
